In [436]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [444]:
from Kickit_app_transformers import Kickit_dataframe_preprocessor
from Kickit_app_transformers import Kickit_frequency_encoder
from Kickit_app_transformers import Kickit_weight_of_evidence_encoder
from Kickit_app_transformers import  Kickit_dataframe_balancer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from Kickit_app_transformers import woe_cat_dict
import warnings
import pickle
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesRegressor
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV

# Reading the Data set

In [445]:
# reading the dataset
df_raw = pd.read_csv('kick.csv')
df_raw.head(2)

,IsBadBuy,PurchDate,Auction,VehYear,VehicleAge,Make,Model,Trim,SubModel,Color,...,MMRCurrentRetailAveragePrice,MMRCurrentRetailCleanPrice,PRIMEUNIT,AUCGUART,BYRNO,VNZIP1,VNST,VehBCost,IsOnlineSale,WarrantyCost
0,0,1260144000,ADESA,2006,3,MAZDA,MAZDA3,i,4D SEDAN I,RED,...,11597,12409,?,?,21973,33619,FL,7100,0,1113
1,0,1260144000,ADESA,2004,5,DODGE,1500 RAM PICKUP 2WD,ST,QUAD CAB 4.7L SLT,WHITE,...,11374,12791,?,?,19638,33619,FL,7600,0,1053


Extracting the axis of studies for the project custom to  ADESA website 

In [446]:
choosen_axis = ["Model", "Make" ,"Transmission","VehOdo","MMRAcquisitionAuctionAveragePrice", "IsBadBuy"]
df_adesa_raw = df_raw[choosen_axis]
df_adesa_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72983 entries, 0 to 72982
Data columns (total 6 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Model                              72983 non-null  object
 1   Make                               72983 non-null  object
 2   Transmission                       72983 non-null  object
 3   VehOdo                             72983 non-null  int64 
 4   MMRAcquisitionAuctionAveragePrice  72983 non-null  object
 5   IsBadBuy                           72983 non-null  int64 
dtypes: int64(2), object(4)
memory usage: 3.3+ MB


In [447]:
#df_adesa_raw['MMRAcquisitionAuctionAveragePrice'] = df_adesa_raw['MMRAcquisitionAuctionAveragePrice'].replace(['?'], '10')

In [448]:
#df_adesa_raw['MMRAcquisitionAuctionAveragePrice'].astype('str')

In [449]:
#df_adesa_raw['MMRAcquisitionAuctionAveragePrice'] = df_adesa_raw['MMRAcquisitionAuctionAveragePrice'].astype('float')

# Pre-processing Pipeline for the freqency encoding technique

copying the raw data

In [450]:
df = df_adesa_raw.copy()

 Instantiating the preprocessing pipeline objects

In [451]:
# this on will clean the data with respect to our strategy
f_preprocess = Kickit_dataframe_preprocessor()
# this one will balance the dataset
f_balancer = Kickit_dataframe_balancer()
# this one will use the frequency encoding technique for categorical variable 
freq_encoder = Kickit_frequency_encoder()

Creating a pipeline for the first preprocessing technique :based on count of  frequency encoder

In [452]:
freq_pipeline = Pipeline([("preprocess",f_preprocess),("balancer", f_balancer),
                          ("fencoder",freq_encoder)])

preprocessing the dataframe with the count of frequency pipeline

In [453]:
adesa_freq_pipe = freq_pipeline.fit_transform(df)

checking the transformation for success

In [454]:
adesa_freq_pipe.head(5)

,Model,Make,Transmission,VehOdo,MMRAcquisitionAuctionAveragePrice,IsBadBuy
0,42,269,17360,89046,8155.0,0
1,236,2947,17360,93593,6854.0,0
2,139,2947,17360,73807,3202.0,0
3,136,2947,17360,65617,1893.0,0
4,246,3015,591,69367,3913.0,0


In [455]:
adesa_freq_pipe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17952 entries, 0 to 17951
Data columns (total 6 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Model                              17952 non-null  int64  
 1   Make                               17952 non-null  int64  
 2   Transmission                       17952 non-null  int64  
 3   VehOdo                             17952 non-null  int64  
 4   MMRAcquisitionAuctionAveragePrice  17952 non-null  float64
 5   IsBadBuy                           17952 non-null  int64  
dtypes: float64(1), int64(5)
memory usage: 841.6 KB


###  Modelization based on frequency count

 preparing and spliting the dataset for the modelization

In [456]:
features = adesa_freq_pipe.drop("IsBadBuy", axis =1)
target = adesa_freq_pipe["IsBadBuy"]
x_ftrain, x_ftest, y_ftrain, y_ftest = train_test_split(features, target, test_size=0.25, random_state=0)

Creating a list of pipelines for three different candidate classifiers 

In [457]:
f_scaler = StandardScaler()
f_randomForest = RandomForestClassifier()
f_logisticRegr = LogisticRegression(random_state=0)
f_decisionTree = DecisionTreeClassifier()

freq_bench_cross = [Pipeline([("scaler",f_scaler), ('lr_classifier', f_logisticRegr)]),
                    Pipeline([('dt_classifier', f_decisionTree)]),
                    Pipeline([('rf_classifier', f_randomForest)])] 

In [458]:
best_faccuracy=0.0
best_fclassifier=0
best_fpipeline=""

In [459]:
# Dictionary of pipelines and classifier types for ease of reference
freq_pipe_dict = {0: 'Logistic Regression', 1: 'Decision Tree', 2: 'RandomForest'}

# Fit the pipelines
for pipe in freq_bench_cross:
    pipe.fit(x_ftrain, y_ftrain)

In [460]:
for i,model in enumerate(freq_bench_cross):
    print("{} with count of frequency Test Accuracy: {}".format(freq_pipe_dict[i],model.score(x_ftest,y_ftest)))

Logistic Regression with count of frequency Test Accuracy: 0.5877896613190731
Decision Tree with count of frequency Test Accuracy: 0.5521390374331551
RandomForest with count of frequency Test Accuracy: 0.5935828877005348


# Pipelines for the Weight of Evidence  encoding technique

### Preprocessing the dataframe with the weight of evidence pipeline

In [461]:
df = df_adesa_raw.copy()

 Instantiating the pipeline object

In [462]:
# this on will clean the data with respect to our requirements
w_preprocess = Kickit_dataframe_preprocessor()
# this one will balance the dataset
w_balancer = Kickit_dataframe_balancer()
# # this one will use the weight of evidence encoding technique for categorical variable 
woe_encoder = Kickit_weight_of_evidence_encoder()

#Creating a pipeline for the second  preprocessing technique :based on the weight of evidence encoder

In [463]:
woe_pipeline = Pipeline([("preprocess",w_preprocess),("balancer", w_balancer),
                         ("woe_encoder", woe_encoder)])

preprocessing the dataframe with the weight of evidence pipeline

In [467]:
# the next command  after this will raise an unecessary warning message, this will help keep the notebook clean
warnings.filterwarnings('ignore')

In [468]:
adesa_woe_pipe = woe_pipeline.fit_transform(df)

In [469]:
# checking results
adesa_woe_pipe.head(5)

,Model,Make,Transmission,VehOdo,MMRAcquisitionAuctionAveragePrice,IsBadBuy
0,0.376478,-0.250261,0.026150,89046,8155.0,0
1,-0.169179,0.194268,0.026150,93593,6854.0,0
2,0.480463,0.194268,0.026150,73807,3202.0,0
3,-0.601439,0.194268,0.026150,65617,1893.0,0
4,-0.547706,-0.286205,0.010791,69367,3913.0,0


In [470]:
# Checking the dictionary from our encoding class created for each category

In [471]:
woe_cat_dict 
#the nan value are here can simply be compared to having no values or zero, we will set it in the flask code part
# using the np.nan_to_num method

{'1500 RAM PICKUP 2WD': -0.16917873135476025,
 '1500 RAM PICKUP 4WD': -0.3677247801253174,
 '1500 SIERRA PICKUP 2': 0.41985384556026406,
 '1500 SIERRA PICKUP 4': nan,
 '1500 SILVERADO PICKU': 0.742374121325096,
 '1500HD SIERRA PICKUP': nan,
 '2500 RAM PICKUP 2WD': -0.5108256237659907,
 '2500 RAM PICKUP 4WD': nan,
 '2500HD SILVERADO PIC': -0.25131442828090605,
 '3.2 CL 3.2L V 6 NA S': nan,
 '3.2 TL': nan,
 '3.2 TL 3.2L V6 EFI': nan,
 '3.2 TL 3.2L V6 FI DO': nan,
 '300': 0.7810085363512795,
 '300 2.7L V6 MPI': 1.0986122886681098,
 '300 3.5L / 6.0L V12': nan,
 '300 3.5L V6 MPI': nan,
 '300 3.5L V6 MPI / SM': 0.0,
 '300C': 0.0,
 '300C 5.7L V8 SFI': nan,
 '300C 5.7L V8 SFI / S': nan,
 '300M': -1.0986122886681098,
 '300M 3.5L V6 MPI': -0.8472978603872037,
 '300M 3.5L V6 MPI HO': nan,
 '350Z': nan,
 '350Z MFI V6 3.5L DOH': nan,
 '4 RUNNER 2WD V6': nan,
 '4 RUNNER 2WD V8': nan,
 '4 RUNNER 2WD V8 4.7L': nan,
 '4 RUNNER 4WD V6': nan,
 '4 RUNNER 4WD V6 3.4L': nan,
 '626 4C': nan,
 '626 4C 2.0L I-

In [472]:
adesa_woe_pipe.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17381 entries, 0 to 17951
Data columns (total 6 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Model                              17381 non-null  float64
 1   Make                               17381 non-null  float64
 2   Transmission                       17381 non-null  float64
 3   VehOdo                             17381 non-null  int64  
 4   MMRAcquisitionAuctionAveragePrice  17381 non-null  float64
 5   IsBadBuy                           17381 non-null  int64  
dtypes: float64(4), int64(2)
memory usage: 950.5 KB


### Modelization based on WOE 

In [473]:
features = adesa_woe_pipe.drop("IsBadBuy", axis =1)
target = adesa_woe_pipe["IsBadBuy"]
x_wtrain, x_wtest, y_wtrain, y_wtest = train_test_split(features, target, test_size=0.25, random_state=0)

In [474]:
w_scaler = StandardScaler()
w_randomForest = RandomForestClassifier()
w_logisticRegr = LogisticRegression(random_state=0)
w_decisionTree = DecisionTreeClassifier()

# Creating a list of pipelines for three different candidate classifiers 
woe_bench_cross = [Pipeline([("scaler",w_scaler), ('lr_classifier', w_logisticRegr)]),
                    Pipeline([('dt_classifier', w_decisionTree)]),
                    Pipeline([('rf_classifier', w_randomForest)])] 

In [475]:
# Dictionary of pipelines and classifier types for ease of reference
woe_pipe_dict = {0: 'Logistic Regression', 1: 'Decision Tree', 2: ' Default mode RandomForest'}

# Fit the pipelines
for pipe in woe_bench_cross:
    pipe.fit(x_wtrain, y_wtrain)

In [476]:
for i,model in enumerate(woe_bench_cross):
    print("{} with Weight of evidence Test Accuracy: {}".format(woe_pipe_dict[i],model.score(x_wtest,y_wtest)))

Logistic Regression with Weight of evidence Test Accuracy: 0.6415094339622641
Decision Tree with Weight of evidence Test Accuracy: 0.5717901518637828
 Default mode RandomForest with Weight of evidence Test Accuracy: 0.6122871606074551


# Random frorest classifier Auto-Hyperparameters tuning

In [477]:
# Create a pipeline
pipe = make_pipeline((RandomForestClassifier()))
# Create dictionary with candidate learning algorithms and their hyperparameters
grid_param = [
                {"randomforestclassifier": [RandomForestClassifier()],
                 "randomforestclassifier__n_estimators": [10],
                 "randomforestclassifier__max_depth":[10,30,25,20],
                 "randomforestclassifier__min_samples_leaf":[1,2,5,6,10,15,100],
                 "randomforestclassifier__max_leaf_nodes": [2, 5,10]}]
# create a gridsearch of the pipeline, the fit the best model
gridsearch = GridSearchCV(pipe, grid_param, cv=5, verbose=0,n_jobs=-1) # Fit grid search
Upgraded_Random_frorest = gridsearch.fit(x_wtrain,y_wtrain)

In [478]:
Upgraded_Random_frorest.score(x_wtest,y_wtest)

0.6412793373216751

In [479]:
print(Upgraded_Random_frorest.best_estimator_)

Pipeline(steps=[('randomforestclassifier',
                 RandomForestClassifier(max_depth=25, max_leaf_nodes=10,
                                        min_samples_leaf=10,
                                        n_estimators=10))])


# Conclusions

The best pipeline between the frequency encoding and the weight of evidence encoding is  : Weight of evidence one 

The best model within that pipeline is : Logistic regression  then Upgraded Random forest models with the same score

saving the created encoding dictionnary for interpretation in the flask part

In [488]:
file = open('kidict.pkl', 'wb')
pickle.dump(woe_cat_dict, file)

saving the selected model

In [490]:
file = open('model.pkl', 'wb')
pickle.dump(Upgraded_Random_frorest, file)